# Twitter Pre-processing

Twitter Pre-processing to build following:

1. Twitter Timeline Data
2. Twitter User Profile Data
3. Twitter Most Frequent Words Data
4. Twitter Most Frequent Hashtags Data
5. Twitter Most Frequent Mentions Data

In [72]:
import json
import requests
from requests_oauthlib import OAuth1
import json
from urllib.parse import urlparse
import matplotlib.pyplot as plt
import tweepy
import seaborn as sns
%matplotlib inline
import os
import pandas as pd
import seaborn as sns
from pathlib import Path

plt.style.use('seaborn')

In [98]:
campus = "iupui"
twitter_file = "indiana-university_purdue-university-indianapolis"

## Settings

In [99]:
raw_data_dir = f"../../data/raw_data/{campus}/twitter/"
raw_file_data = f"{twitter_file}_twitter_data.csv"
raw_file_userinfo = f"{twitter_file}_twitter_user_profile_data.csv"


processed_data_dir =  f"../../data/processed_data/{campus}/twitter/"
file = f"{twitter_file}_twitter_data.xlsx"


####

### twitter data - main sheet
twitter_data_sheetname = "twitter_data"

## user profile sheet 
twitter_user_profile_data_sheetname = "user_profile_data"

## Most frequent words
most_frequent_words_sheetname = "most_freq_100_words"

## Most frequent hashtags
most_frequent_hashtags_sheetname = "most_freq_100_hashtags"

## Most frequent screennames
most_frequent_screennames_sheetname = "most_freq_100_mentions"

###



In [100]:
import nltk
stop_words =   nltk.corpus.stopwords.words('english') + [
 '.',
 ',',
 '--',
 '\'s',
 '?',
 ')',
 '(',
 ':',
 '\'',
 '\'re',
 '"',
 '-',
 '}',
 '{',
 u'—',
     
 ]

In [101]:
## Create Directory if it doesnt exists
def make_dir_if_not_exists(directory):
    dir_path = Path(directory)
    if not dir_path.exists():
        os.makedirs(dir_path)
    else:
        print("Path exists")
    
make_dir_if_not_exists(processed_data_dir)
    

Path exists


## Read Data

In [102]:
import shutil  

## raw file
#shutil.copyfile(os.path.join(raw_data_dir,raw_file_data), os.path.join(processed_data_dir,raw_file_data))
#print("Copied data file!")
#shutil.copyfile(os.path.join(raw_data_dir,raw_file_userinfo), os.path.join(processed_data_dir,raw_file_userinfo))
#print("Copied user file!")



In [103]:
## read data
import csv
df = pd.read_csv(os.path.join(raw_data_dir,raw_file_data))
user_data_df = pd.read_csv(os.path.join(raw_data_dir,raw_file_userinfo))

## save currentfile 
#df.to_excel(os.path.join(processed_data_dir,"indiana-university-bl_twitter_data.xls"),index=False)

df["created"] = pd.to_datetime(df["created"])

In [104]:
print("No of rows and columns: {}, min date: {}, max date: {}".format(df.shape,df['created'].min(),df['created'].max()))
      

No of rows and columns: (2537, 22), min date: 2018-07-01 00:01:57, max date: 2019-07-26 18:02:08


In [105]:
df.head()

,created,text,sentiment,tweet_id,truncated,user,user_followers_count,user_friends_count,user_favourites_count,user_statuses_count,...,favorite_count,reply_count,at_mentions,hashtags,urls,source,lat,long,tweet_type,words
0,2019-07-26 18:02:08,"Just one more month. See you soon, Jags. ❤️ https://t.co/ED6xnQB5Ti",Neutral,1154814171478974464,False,IUPUI,35948,3151,5144,27089,...,111,0,NaN,NaN,NaN,"<a href=""https://about.twitter.com/products/tweetdeck"" rel=""nofollow"">TweetDeck</a>",NaN,NaN,tweet,one month see soon jag ❤ ️
1,2019-07-26 15:11:48,"RT @IUBResearch: RT @IUImpact: You vacuum it up, sweep it up and wipe it off your furniture. But do you know what it actually is - and how…",Negative,1154771305608228865,False,IUPUI,35948,3151,5144,27089,...,0,0,"IUBResearch, IUImpact",NaN,NaN,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",NaN,NaN,retweet,vacuum sweep wipe furnitur know actual …
2,2019-07-26 15:05:06,Looking for things to do in Indy this weekend? @VisitIndy has you covered: https://t.co/3o1kYy7JMe,Neutral,1154769617484689408,False,IUPUI,35948,3151,5144,27089,...,2,0,VisitIndy,NaN,https://t.co/3o1kYy7JMe,"<a href=""https://sproutsocial.com"" rel=""nofollow"">Sprout Social</a>",NaN,NaN,tweet,look thing indi weekend cover
3,2019-07-25 19:35:21,Want a chance to win an IUPUI beach towel? Follow this link to tell Chancellor @paydar what you're most excited abo… https://t.co/Oipm5e8CO3,Positive,1154475241982156800,True,IUPUI,35948,3151,5144,27089,...,10,0,paydar,NaN,https://t.co/Oipm5e8CO3,"<a href=""https://mobile.twitter.com"" rel=""nofollow"">Twitter Web App</a>",NaN,NaN,quote,want chanc win iupui beach towel follow link tell chancellor excit abo …
4,2019-07-25 15:37:56,Professor Hawkins is ready for the fowl semester 🧐 https://t.co/L7Q34Ocvjc,Neutral,1154415496151932929,False,IUPUI,35948,3151,5144,27089,...,39,0,NaN,NaN,https://t.co/L7Q34Ocvjc,"<a href=""https://about.twitter.com/products/tweetdeck"" rel=""nofollow"">TweetDeck</a>",NaN,NaN,quote,professor hawkin readi fowl semest


In [106]:
pd.set_option('display.max_colwidth',-1)



## Most Frequent Words


In [107]:
from nltk.tokenize import TweetTokenizer
from nltk.stem.porter import PorterStemmer

def tweet_tokenizer(txt):
    try:
        tokenizer = TweetTokenizer()
        all_tokens = tokenizer.tokenize(txt.lower())
        # this line filters out all tokens that are entirely non-alphabetic characters
        filtered_tokens = [t for t in all_tokens if t.islower()]
        # filter out all tokens that are <=2 chars
        filtered_tokens = [x for x in filtered_tokens if len(x)>2]
    except IndexError:
        filtered_tokens = []
    return(filtered_tokens)


tweets_text = str(df["text"])

In [108]:
tweet_tokenizer(tweets_text)

['just',
 'one',
 'more',
 'month',
 'see',
 'you',
 'soon',
 'jags',
 'https://t.co/ed6xnqb5ti',
 '@iubresearch',
 '@iuimpact',
 'you',
 'vacuum',
 'sweep',
 'and',
 'wipe',
 'off',
 'your',
 'furniture',
 'but',
 'you',
 'know',
 'what',
 'actually',
 'and',
 'how',
 'looking',
 'for',
 'things',
 'indy',
 'this',
 'weekend',
 '@visitindy',
 'has',
 'you',
 'covered',
 'https://t.co/3o1kyy7jme',
 'want',
 'chance',
 'win',
 'iupui',
 'beach',
 'towel',
 'follow',
 'this',
 'link',
 'tell',
 'chancellor',
 '@paydar',
 'what',
 "you're",
 'most',
 'excited',
 'abo',
 'https://t.co/oipm5e8co3',
 'professor',
 'hawkins',
 'ready',
 'for',
 'the',
 'fowl',
 'semester',
 'https://t.co/l7q34ocvjc',
 'another',
 'doggone',
 'beautiful',
 'morning',
 'indy',
 'https://t.co/ffs9qtvo4f',
 'attention',
 'https://t.co/5xlps86taz',
 "it's",
 'the',
 'air',
 'and',
 'all',
 'around',
 'but',
 'what',
 'dust',
 'one',
 'iupui',
 'researcher',
 'analyzing',
 'samples',
 'collected',
 'from',
 'peop',

In [109]:
## Get most frequent words
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('stopwords')

def get_most_frequent_words(text_series, stop_words = None, ngram_range = (1,2)):
    
    count_vectorizer = CountVectorizer(analyzer = "word",
                                       tokenizer = tweet_tokenizer,
                                       stop_words = stop_words,
                                       ngram_range = ngram_range 
                                      )
    
    term_freq_matrix = count_vectorizer.fit_transform(text_series)
        
    terms = count_vectorizer.get_feature_names()
    term_frequencies = term_freq_matrix.sum(axis = 0).tolist()[0]
    
    
    term_freq_df = (pd.DataFrame(list(zip(terms, term_frequencies)), columns = ["token","count"])
                    .set_index("token")
                    .sort_values("count",ascending = False))
    
    ### Avoiding hashtags and mentions
    data = term_freq_df.index.str.startswith(("@","#"))

    data = [not(data[i]) for i in range(len(data))]
    term_freq_df = term_freq_df[data]
    return term_freq_df

term_freq_df = get_most_frequent_words(df["text"],stop_words = stop_words)


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mchivuku/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [110]:
term_freq_df = term_freq_df[:100]

## Most Frequent Hashtags and Screennames

In [111]:
df.columns

Index(['created', 'text', 'sentiment', 'tweet_id', 'truncated', 'user',
       'user_followers_count', 'user_friends_count', 'user_favourites_count',
       'user_statuses_count', 'retweeted_status', 'retweet_count',
       'favorite_count', 'reply_count', 'at_mentions', 'hashtags', 'urls',
       'source', 'lat', 'long', 'tweet_type', 'words'],
      dtype='object')

In [112]:
hashtags = []
screen_names = []
for index, row in df.iterrows():
    if isinstance(row["hashtags"], str):
        hashtags.extend([ t for t in str(row["hashtags"]).split(", ")])
    if isinstance(row["at_mentions"], str):
        screen_names.extend([t for t in str(row["at_mentions"]).split(", ")]) 
    


In [113]:
print("Hashtags: ", len(hashtags),"Screen_names:", len(screen_names))


Hashtags:  581 Screen_names: 2514


#### Build Hashtag and Mentions table

In [114]:
from prettytable import PrettyTable
from collections import Counter
for label, data in (('Screen Name', screen_names),
                    ('Hashtag', hashtags)):
    pt = PrettyTable(field_names=[label, 'Count'])
    
    
    ### Merge duplicates
    c = {}
   
    for k in list(data):
        keys = [str.lower(p) for p in c.keys()]
        if str.lower(k) in keys:
            current_keys = list(c.keys())
            match_index = keys.index(str.lower(k))
            match = current_keys[match_index]
            c[match] += 1
        else:
            c[k] = 1
    
    if label == 'Screen Name':
        screenname_df = pd.DataFrame.from_dict(c, orient='index').reset_index()
        screenname_df = screenname_df.rename(columns={'index':'screen_name', 0:'count'}).sort_values("count",ascending = False)
        print(screenname_df.head())
    else:
        hashtag_df = pd.DataFrame.from_dict(c, orient='index').reset_index()
        hashtag_df = hashtag_df.rename(columns= {'index':"hashtag",0:'count'}).sort_values("count",ascending = False)
        print(hashtag_df.head())
    

         screen_name  count
5    IUPUI            377  
3    paydar           229  
102  IUPUIJaguars     60   
109  IndianaUniv      54   
105  IUPUICampusCntr  43   
             hashtag  count
14   MyIUPUI          43   
20   JagsROAR         36   
25   iupui            26   
38   iupuigrad19      20   
175  YouGottaRegatta  18   


In [115]:
hashtag_df.head()

,hashtag,count
14,MyIUPUI,43
20,JagsROAR,36
25,iupui,26
38,iupuigrad19,20
175,YouGottaRegatta,18


In [116]:
print(f"Screennames: {screenname_df.shape}, hashtags: {hashtag_df.shape}")

Screennames: (685, 2), hashtags: (221, 2)


### Top 100 - mentions and hashtags

In [117]:
screenname_df, hashtag_df  = screenname_df[:100], hashtag_df[:100]

### Add Created Date


In [118]:
df['created date'] = df["created"].dt.date


## Build Excel Sheets

In [119]:
sentiment_df = df.loc[:,['sentiment']]


In [120]:
#df = df.drop(['Created Date'],axis=1)
df.head()

,created,text,sentiment,tweet_id,truncated,user,user_followers_count,user_friends_count,user_favourites_count,user_statuses_count,...,reply_count,at_mentions,hashtags,urls,source,lat,long,tweet_type,words,created date
0,2019-07-26 18:02:08,"Just one more month. See you soon, Jags. ❤️ https://t.co/ED6xnQB5Ti",Neutral,1154814171478974464,False,IUPUI,35948,3151,5144,27089,...,0,NaN,NaN,NaN,"<a href=""https://about.twitter.com/products/tweetdeck"" rel=""nofollow"">TweetDeck</a>",NaN,NaN,tweet,one month see soon jag ❤ ️,2019-07-26
1,2019-07-26 15:11:48,"RT @IUBResearch: RT @IUImpact: You vacuum it up, sweep it up and wipe it off your furniture. But do you know what it actually is - and how…",Negative,1154771305608228865,False,IUPUI,35948,3151,5144,27089,...,0,"IUBResearch, IUImpact",NaN,NaN,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",NaN,NaN,retweet,vacuum sweep wipe furnitur know actual …,2019-07-26
2,2019-07-26 15:05:06,Looking for things to do in Indy this weekend? @VisitIndy has you covered: https://t.co/3o1kYy7JMe,Neutral,1154769617484689408,False,IUPUI,35948,3151,5144,27089,...,0,VisitIndy,NaN,https://t.co/3o1kYy7JMe,"<a href=""https://sproutsocial.com"" rel=""nofollow"">Sprout Social</a>",NaN,NaN,tweet,look thing indi weekend cover,2019-07-26
3,2019-07-25 19:35:21,Want a chance to win an IUPUI beach towel? Follow this link to tell Chancellor @paydar what you're most excited abo… https://t.co/Oipm5e8CO3,Positive,1154475241982156800,True,IUPUI,35948,3151,5144,27089,...,0,paydar,NaN,https://t.co/Oipm5e8CO3,"<a href=""https://mobile.twitter.com"" rel=""nofollow"">Twitter Web App</a>",NaN,NaN,quote,want chanc win iupui beach towel follow link tell chancellor excit abo …,2019-07-25
4,2019-07-25 15:37:56,Professor Hawkins is ready for the fowl semester 🧐 https://t.co/L7Q34Ocvjc,Neutral,1154415496151932929,False,IUPUI,35948,3151,5144,27089,...,0,NaN,NaN,https://t.co/L7Q34Ocvjc,"<a href=""https://about.twitter.com/products/tweetdeck"" rel=""nofollow"">TweetDeck</a>",NaN,NaN,quote,professor hawkin readi fowl semest,2019-07-25


<ul>
<li> Twitter Data </li>
<li> User Profile Data </li>
<li> Most Frequent Words </li>
<li> Most Frequent Hashtags </li>
<li> Most Frequent Mentions </li>

</ul>

In [121]:
user_data_df

,user_id,name,screen_name,followers_count,friends_count,favourites_count,statuses_count,total_reach,avg_follower_cnt
0,10086382,IUPUI,IUPUI,35948,3151,5144,27089,10395659,289.19


In [122]:

with pd.ExcelWriter(os.path.join(processed_data_dir,file),engine='xlsxwriter') as writer:
    df.to_excel(writer, sheet_name=twitter_data_sheetname,index=False)
    user_data_df.to_excel(writer, sheet_name=twitter_user_profile_data_sheetname,index=False)
    term_freq_df.to_excel(writer, sheet_name=most_frequent_words_sheetname)
    screenname_df.to_excel(writer, sheet_name=most_frequent_screennames_sheetname,index=False)
    hashtag_df.to_excel(writer, sheet_name=most_frequent_hashtags_sheetname,index=False)
    writer.save()
    